# Keras exercise

In this exercise you will be creating a Keras model by loading a data set, preprocessing input data, building a Sequential Keras model and compiling the model with a training configuration. Afterwards, you train your model on the training data and evaluate it on the test set. To finish this exercise, you will then save your model in HDF5 format.

## Data

For this exercise we will use the Reuters newswire dataset. This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes, just as in the IMDB data we encountered in lecture 5 of this series. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

## Goal

We want to create a Multi-layer perceptron (MLP) using Keras which we can train to classify news items into the specified 46 topics.

## Instructions

We start by importing everything we need for this exercise:

In [27]:
import pip

try:
    __import__('keras')
except ImportError:
    pip.main(['install', 'keras']) 
    
try:
    __import__('h5py')
except ImportError:
    pip.main(['install', 'h5py']) 

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

seed = 1337
np.random.seed(seed)

IMPORTANT! => In case h5py has been installed, please restart the kernel by clicking on "Kernel"->"Restart and Clear Outout" and wait until all output disapears. Then your changes are beeing picked up

As you can see, we use Keras' Sequential model with only two types of layers: Dense and Dropout. We also specify a random seed to make our results reproducible. Next, we load the Reuters data set:

In [28]:
from keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

Note that we cap the maximum number of words in a news item to 1000 by specifying the *num_words* key word. Also, 20% of the data will be test data and we ensure reproducibility by setting our random seed.

Our training features are still simply sequences of indexes and we need to further preprocess them, so that we can plug them into a *Dense* layer. For this we use a *Tokenizer* from Keras' text preprocessing module. This tokenizer will take an index sequence and map it to a vector of length *max_words=1000*. Each of the 1000 vector positions corresponds to one of the words in our newswire corpus. The output of the tokenizer has a 1 at the i-th position of the vector, if the word corresponding to i is in the description of the newswire, and 0 otherwise. Even if this word appears multiple times, we still just put a 1 into our vector, i.e. our tokenizer is binary. We use this tokenizer to transform both train and test features:

In [29]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

## 1. Exercise part: label encoding

Use to_categorical, as we did in the lectures, to transform both *y_train* and *y_test* into one-hot encoded vectors of length *num_classes*:

In [30]:
# y_train = to_categorical(###_YOUR_CODE_GOES_HERE_###)
# y_test = to_categorical(###_YOUR_CODE_GOES_HERE_###)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

## 2. Exercise part: model definition

Next, initialise a Keras *Sequential* model and add three layers to it:

    Layer: Add a *Dense* layer with in input_shape=(max_words,), 512 output units and "relu" activation.
    Layer: Add a *Dropout* layer with dropout rate of 50%.
    Layer: Add a *Dense* layer with num_classes output units and "softmax" activation.

In [31]:
# model = ###_YOUR_CODE_GOES_HERE_###  # Instantiate sequential model
# model.add(###_YOUR_CODE_GOES_HERE_###) # Add first layer. Make sure to specify input shape
# model.add(###_YOUR_CODE_GOES_HERE_###) # Add second layer
# model.add(###_YOUR_CODE_GOES_HERE_###) # Add third layer

model = Sequential()
model.add(Dense(
                units = 512,
                input_shape = (max_words,),
                activation = 'relu')) # Add first layer. Make sure to specify input shape
model.add(Dropout(0.5)) # Add second layer
model.add(Dense(
                units = num_classes,
                activation = 'softmax'))# Add third layer

## 3. Exercise part: model compilation

As the next step, we need to compile our Keras model with a training configuration. Compile your model with "categorical_crossentropy" as loss function, "adam" as optimizer and specify "accuracy" as evaluation metric. NOTE: In case you get an error regarding h5py, just restart the kernel and start from scratch

In [32]:
# model.compile(###_YOUR_CODE_GOES_HERE_###)
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
#some learners constantly reported 502 errors in Watson Studio. 
#This is due to the limited resources in the free tier and the heavy resource consumption of Keras.
#This is a workaround to limit resource consumption

from keras import backend as K

K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))


## 4. Exercise part: model training and evaluation

Next, define the batch_size for training as 32 and train the model for 5 epochs on *x_train* and *y_train* by using the *fit* method of your model. Then calculate the score for your trained model by running *evaluate* on *x_test* and *y_test* with the same batch size as used in *fit*.

In [34]:
# batch_size = ###_YOUR_CODE_GOES_HERE_###
# model.fit(###_YOUR_CODE_GOES_HERE_###)
# score = model.evaluate(###_YOUR_CODE_GOES_HERE_###)
batch_size = 32
model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=5)
score = model.evaluate(x_test, y_test)

Epoch 1/5
8982/8982 [==============================] - 6s 676us/step - loss: 1.3942 - acc: 0.6905
Epoch 2/5
8982/8982 [==============================] - 6s 615us/step - loss: 0.7735 - acc: 0.8176
Epoch 3/5
8982/8982 [==============================] - 5s 610us/step - loss: 0.5569 - acc: 0.8641 0s - loss: 0.5669 - acc
Epoch 4/5
8982/8982 [==============================] - 5s 606us/step - loss: 0.4223 - acc: 0.8937
Epoch 5/5
2246/2246 [==============================] - 0s 108us/step


If you have done everything as specified, in particular set the random seed as we did above, your test accuracy should be around 80% 

In [35]:
score[1]

0.7991985752448798



## 5. Exercise part: model serialisation

As the final step in this exercise, save your model as "model.h5" and upload this file for evaluation:

In [36]:
model.save("model.h5")  # upload this file to the grader in the next code block

To upload the exported model to the grader we first need to encode it based64, we are doing this using a shell command:

In [40]:
import base64

In [41]:
!base64 model.h5 > model.h5.base64

'base64' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
# echo %OPENSSL_CONF%

# set OPENSSL_CONF=C:\OpenSSL\bin\openssl.cnf

In [18]:
!certutil -encode model.5h tmp.b64 && findstr /v /c:- tmp.b64 > model.5h.b64

DecodeFile returned The system cannot find the file specified. 0x80070002 (WIN32: 2 ERROR_FILE_NOT_FOUND)
CertUtil: -encode command FAILED: 0x80070002 (WIN32: 2 ERROR_FILE_NOT_FOUND)
CertUtil: The system cannot find the file specified.


In [37]:
!certutil -encode C:\Users\ming\Documents\Backup_G\Advanced_Data_Science_with_IBM\model.h5 > C:\Users\ming\Documents\Backup_G\Advanced_Data_Science_with_IBM\model.h5.base64

In [16]:
!base64 -i <model.5h> -o <model.5h.base64>

The syntax of the command is incorrect.


In [13]:
!openssl base64 -in model.5h -output model.5h.base64

base64: Unrecognized flag output
base64: Use -help for summary.


In [11]:
!openssl enc -base64 -in model.5h -out model.5h.base64

Can't open model.5h for reading, No such file or directory
2520:error:02001002:system library:fopen:No such file or directory:crypto\bio\bss_file.c:72:fopen('model.5h','rb')
2520:error:2006D080:BIO routines:BIO_new_file:no such file:crypto\bio\bss_file.c:79:


Then we have to install a little library in order to submit to coursera

In [15]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2019-03-26 19:41:27--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com... 151.101.52.133
Connecting to raw.githubusercontent.com|151.101.52.133|:443... connected.
ERROR: cannot verify raw.githubusercontent.com's certificate, issued by `/C=US/O=DigiCert Inc/OU=www.digicert.com/CN=DigiCert SHA2 High Assurance Server CA':
  Unable to locally verify the issuer's authority.
ERROR: certificate common name `www.github.com' doesn't match requested host name `raw.githubusercontent.com'.
To connect to raw.githubusercontent.com insecurely, use `--no-check-certificate'.


In [38]:
import requests, json

def submit(submitterEmail,secret,key,submission_part, all_parts, data):
        submission = {
                    "assignmentKey": key,
                    "submitterEmail":  submitterEmail,
                    "secret":  secret,
                    "parts": {}
                  }
        for part in all_parts:
            if part == submission_part:
                submission["parts"][part] = {"output": data}
            else:
                submission["parts"][part] = dict()
        response = requests.post('https://www.coursera.org/api/onDemandProgrammingScriptSubmissions.v1', data=json.dumps(submission))
        if response.status_code == 201:
            print ("Submission successful, please check on the coursera grader page for the status")
        else:
            print ("Something went wrong, please have a look at the reponse of the grader")
            print ("-------------------------")
            print (response.text)
            print ("-------------------------")
            

Please provide your email address and obtain a submission token (secret) on the grader’s submission page in coursera, then execute the cell

In [39]:
# from rklib import submit

key = "XbAMqtjdEeepUgo7OOVwng"
part = "LqPRQ"
email = "lei.ming296@gmail.com"
secret = "PVc2QY5aU38er4H0"

with open('model.h5.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
